**DISCLAIMER: The author of this notebook and software is in no case liable for any claim, damage or liability that occurs in connection with dealings of the same. Typos, printing-, software- and typesetting errors in this work reserved. The author does not warrant correctness of software or statements made in this work. This notebook is hosted on GitHub solely for the purpose of presentation.**


# Using AI to create AI
## Building a model for very accurate predictions

In this notebook, novel methods of data analysis are used to determine whether breast cancer is malign or benign based on tabular data. Special consideration is given to the fact that only little data is available. This is especially problematic since this leads to high variance when testing models, potentially giving the impression of having a far better (or worse) model at hand than is actually the case.

This problem can be (somewhat expensively) taken care of by using K-fold cross-validation. With datasets as small as this one, it is easy to create stratified splits, meaning that each split more or less has the same class distribution. This will have a positive effect on training and reduce the variance between test scores across the splits.

(If you run this code, always monitor memory usage, as you can damage your hardware otherwise! There should not be too much written out onto the disk — in other words to a page file.) 

The following few code cells will 'set the scene' by defining a few methods that will be used in the course of the notebook. Feel free to skip ahead!

## Dataset
The dataset used for this Notebook is from https://www.kaggle.com/datasets/vijayaadithyanvg/breast-cancer-prediction as seen on 3. Nov. 2022, where it states that it is under the license: "CC0: Public Domain" https://creativecommons.org/publicdomain/zero/1.0/

## Set up (Skip this chapter)

In [1]:
# Config

import tracemalloc

from optuna.trial import FrozenTrial

# Note: This notebook was run with device mps and cpu. The current nightly build of pytorch lead to very high memory consumption with device 'mps'.
# Especially if you change this setting always monitor memory usage, as you can damage your hardware otherwise!
DEVICE = "cpu"  # cpu is actually faster since this tabular data would need more time copying/converting than calculating
RESPONSE_COLUMN = 'diagnosis'
OPTUNA_DB = "sqlite:///optuna_breast_cancer_detection.db"
NUM_FOLDS = 5
TRAIN_PERC = 0.8
TENSORBOARD_LOGDIR_LASSO = "breast_cancer_detection_lasso"
TENSORBOARD_LOGDIR_NN_SHALLOW = "breast_cancer_detection_nn_shallow"
TENSORBOARD_LOGDIR_NN_DROPOUT = "breast_cancer_detection_nn_dropout"
TRAIN_FOR_EPOCHS = 10

WAIT = False
TRACK_MEMORY = False

In [2]:
if TRACK_MEMORY:
    tracemalloc.start()

In [3]:
from typing import List, Tuple, Dict, Union, Callable, Optional


# Measure

def f1_score(ground_truth: List[int], predictions: List[int]):
    fp: int = 0
    fn: int = 0
    tp: int = 0
    tn: int = 0

    for p, t in zip(ground_truth, predictions):
        assert (p in (-1, 1) and t in (-1, 1)) or (p in (0, 1) and t in (0, 1))

        if p == t:
            if p == 1:
                tp += 1
            else:
                tn += 1
        else:
            if p == 1:
                fp += 1
            else:
                fn += 1
    if (2 * tp + fp + fn) == 0:
        return 0
    f1 = (2 * tp)/(2 * tp + fp + fn)
    assert 0 <= f1 <= 1, f1
    return f1

In [4]:
# Methods for accessing and manipulating data

import pandas as pd

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


def get_full_dataframe() -> pd.DataFrame:
    return pd.read_csv('data.csv')


def apply_threshold_on_response(response: List[float], to_zero) -> List[float]:
    if to_zero:
        sub = 0
    else:
        sub = -1
    return [sub if r < 0 else 1 for r in response]


def split_to_train_test(df: pd.DataFrame, train_ratio: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    lengths = [int(len(df) * p) for p in (train_ratio, 1 - train_ratio)]
    lengths[0] += len(df) - sum(lengths)

    tmp_x, tmp_y = split_to_xy(df)
    x_tr: pd.DataFrame
    y_tr: pd.DataFrame
    x_te: pd.DataFrame
    y_te: pd.DataFrame
    x_tr, x_te, y_tr, y_te = train_test_split(tmp_x, tmp_y, stratify=tmp_y[RESPONSE_COLUMN],
                                                        test_size=1 - train_ratio, random_state=0)
    return x_tr.join(y_tr[RESPONSE_COLUMN]), x_te.join(y_te[RESPONSE_COLUMN])


def split_to_xy(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    tmp_x = df.copy()
    tmp_y = pd.DataFrame(tmp_x[RESPONSE_COLUMN])
    tmp_x = tmp_x.drop(columns=[RESPONSE_COLUMN])
    tmp_indexes = pd.DataFrame(tmp_x['id'])
    return tmp_x, tmp_indexes.join(tmp_y)


def get_stratified_k_fold_splits(df, num_folds: int) -> Tuple[Tuple[pd.DataFrame, pd.DataFrame]]:
    tmp_x, tmp_y = split_to_xy(df)

    mb: MultiLabelBinarizer = MultiLabelBinarizer(classes=list(tmp_y[RESPONSE_COLUMN].unique()))

    mskf = MultilabelStratifiedKFold(n_splits=num_folds, random_state=0, shuffle=True)

    training_ids: List[List[int]] = []
    test_ids: List[List[int]] = []

    for train, test in mskf.split(tmp_x, mb.fit_transform(list(tmp_y[RESPONSE_COLUMN]))):
        current_train = []
        for t in train:
            current_train.append(tmp_y.iloc[t]['id'])
        training_ids.append(current_train)

        current_test = []
        for t in test:
            current_test.append(tmp_y.iloc[t]['id'])
        test_ids.append(current_test)
    split_datasets = []

    for i in range(num_folds):
        cur_train: pd.DataFrame = df.loc[df['id'].isin(training_ids[i])]
        cur_test: pd.DataFrame = df.loc[~df['id'].isin(training_ids[i])]
        split_datasets.append((cur_train, cur_test))
    return tuple(split_datasets)


def binarize_response(df: pd.DataFrame, to_zero) -> pd.DataFrame:
    tmp = df.copy()
    if to_zero:
        sub = 0
    else:
        sub = -1
    tmp[RESPONSE_COLUMN] = tmp[RESPONSE_COLUMN].apply(lambda x: 1 if (x == 'M' or x == 1) else sub)
    return tmp


def df_no_id(df: pd.DataFrame) -> pd.DataFrame:
    return df.copy().drop(columns=['id'])


def expand_classes(c: int) -> Tuple[int, int]:
    if c in (-1, 0):
        return 0, 1
    return 1, 0

In [5]:
import time


# Misc

def possible_emergency_brake():
    # Do not allow usage of more than 1 GB of memory
    if TRACK_MEMORY and tracemalloc.get_tracemalloc_memory() /1e9 > 1:
        raise Exception('Aborted due to unexpectedly high memory usage')

def view_tensorboard(logdir: str):
    # Note: Viewing other tensorboard requires kernel restart.
    %load_ext tensorboard
    delete_tensorboard_instances()

    %tensorboard --logdir {logdir}  --host localhost

# check what works best for your system
def cpu_cooldown():
    if WAIT:
        time.sleep(0.15)

In [46]:
# Convenience methods for during development
import optuna
import tempfile
# from tensorboard import notebook
import shutil
import os

def delete_tensorboard_instances():
    if os.path.exists(tensorboard_tmp_files_path := os.path.join(tempfile.gettempdir(), '.tensorboard-info')):
        shutil.rmtree(tensorboard_tmp_files_path)
delete_tensorboard_instances()


def delete_study(name: str) -> None:
    try:
        optuna.delete_study(storage=OPTUNA_DB, study_name=name)
        shutil.rmtree('breast_cancer_detection_' + name)
    except KeyError:
        pass
    except Exception as ex:
        raise ex

## Data Exploration

It is bad practice to skip data exploration, but this would take up a lot of this notebook which is really about something else. You can find data exploration to this dataset on the internet. In "real projects" never skip this step!

In [8]:
df_all = get_full_dataframe()
assert len(df_all[RESPONSE_COLUMN].unique()) == 2
df_all

,id,diagnosis,Radius_mean,Texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,21.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


## Set up for training models

Maybe we want to test our models later with different meassures in mind. The results will therefore be categorized according to the measure as well as the modelclass.

In [9]:
# measure, method
measure_dict: Dict[str, callable] = dict()

# measure, model, result
result_dict: Dict[str, Dict[str, float]] = dict()

We could use all data to train but then we would never know how good the model performs in unseen cases. Thus we split a part of the data into a testset, which must only be used once by each model - at the final evaluation that is.

In [10]:
df_train, df_test = split_to_train_test(df_all, TRAIN_PERC)
assert len(df_train) > len(df_test)
print('Length of training data {}\nLength of test data: {}\nTotal length: {}\nRatio train to test: {}\nRatio test to total: {}'.format(len_train := len(df_train), len_test := len(df_test), len_total := len_train + len_test, len_train / len_total, len_test / len_total))

Length of training data 455
Length of test data: 114
Total length: 569
Ratio train to test: 0.7996485061511424
Ratio test to total: 0.20035149384885764


This is close to what was intended. Now the training dataset is split up into many fold. It is important to note that the test dataset will not be part of the folds!

In [11]:
datasets_tvt = get_stratified_k_fold_splits(df_train, num_folds=NUM_FOLDS)

Instead of predicting 'b' or 'm' it makes more sense to predict -1 or +1. The reason why this was chosen over 0 and 1 is that this way a simple sign function can be used to later threshold the response of the model which is a real number. Note that this is only relevant for the lasso regression and later on in the notebook the more conventional 0 and 1 will be used.

In [12]:
tmp_datasets_tvt = []
for i in range(len(datasets_tvt)):
    tmp_datasets_tvt.append((datasets_tvt[i][0], binarize_response(datasets_tvt[i][1], False)))
datasets_tvt = tmp_datasets_tvt
del tmp_datasets_tvt

The sets are split up to inputs and targets

In [13]:
from sklearn.linear_model import Lasso

all_X_train, all_y_train = split_to_xy(df_train)
all_X_test, all_y_test = split_to_xy(df_test)

The F1 score can be used for binary classification tasks

In [14]:
measure_dict['F1'] = f1_score
result_dict['F1'] = dict()

## Baseline model - Lasso regression

Now a lasso model is trained, but what hyper parameters should be chosen? A simple approach would be using a grid search which is essentially just trying out every combination of values from predefined spectrum. This would be hugily inefficient. Enter optuna - This library conducts experiments and learns from previous runs, thus selecting parameters far more efficiently. It can also parameterize dynamically which will be seen later in this notebook. With very little additional effort we can create one of the best possible models in this model class.

In [15]:
from optuna.integration import TensorBoardCallback
from sklearn import preprocessing
scalers = [preprocessing.MinMaxScaler(), preprocessing.RobustScaler(), preprocessing.StandardScaler(), preprocessing.Normalizer()]

tensorboard_callback_lasso = TensorBoardCallback(TENSORBOARD_LOGDIR_LASSO + '/', metric_name="F1")

try:
    study = optuna.load_study(storage=OPTUNA_DB, study_name="lasso_regression")
except KeyError:
    def objective(trial):
        scaler = scalers[trial.suggest_categorical(name='scaler', choices=[0,1,2,3])]

        f1_scores = []
        for lasso_seed, d in enumerate(datasets_tvt):
            k_train_X, k_train_y = split_to_xy(d[0])
            k_train_X = pd.DataFrame(scaler.fit_transform(k_train_X.to_numpy()), columns=k_train_X.columns)
            k_test_X, k_test_y = split_to_xy(d[1])
            k_test_X = pd.DataFrame(scaler.fit_transform(k_test_X.to_numpy()), columns=k_train_X.columns)
            la = Lasso(alpha=trial.suggest_float(name='alpha', low=0.00001, high=0.01, log=True), fit_intercept=trial.suggest_categorical(name='fit_intercept', choices=[True, False]), random_state=lasso_seed)
            la.fit(df_no_id(k_train_X), binarize_response(df_no_id(k_train_y), False))
            f1_scores.append(measure_dict['F1'](list(binarize_response(k_test_y, False)[RESPONSE_COLUMN]), apply_threshold_on_response(list(la.predict(df_no_id(k_test_X))), False)))
        return sum(f1_scores) / len(f1_scores)

    study = optuna.create_study(storage=OPTUNA_DB, study_name="lasso_regression", direction="maximize", sampler=optuna.samplers.TPESampler(seed=0, n_startup_trials=100, multivariate=True, group=True), load_if_exists=True)

    study.optimize(objective, n_trials=150, show_progress_bar=True, callbacks=[tensorboard_callback_lasso])
except Exception as e:
    raise e
study.best_trial

[I 2022-11-03 17:38:38,372] Trial 0 finished with value: 0.942899714536612 and parameters: {'scaler': 1, 'alpha': 0.00018662266976517952, 'fit_intercept': True}. Best is trial 0 with value: 0.942899714536612.
[I 2022-11-03 17:38:39,135] Trial 1 finished with value: 0.8941647597254004 and parameters: {'scaler': 1, 'alpha': 0.0003860866271460546, 'fit_intercept': False}. Best is trial 0 with value: 0.942899714536612.
[I 2022-11-03 17:38:39,587] Trial 2 finished with value: 0.544 and parameters: {'scaler': 3, 'alpha': 0.002160082074140205, 'fit_intercept': False}. Best is trial 0 with value: 0.942899714536612.
[I 2022-11-03 17:38:40,159] Trial 3 finished with value: 0.7872460576835973 and parameters: {'scaler': 0, 'alpha': 0.0008313101133778736, 'fit_intercept': False}. Best is trial 0 with value: 0.942899714536612.
[I 2022-11-03 17:38:40,606] Trial 4 finished with value: 0.0 and parameters: {'scaler': 3, 'alpha': 0.0002335882519483353, 'fit_intercept': True}. Best is trial 0 with value: 

<ipython-input-15-1ea8510a438e>:5: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
  tensorboard_callback_lasso = TensorBoardCallback(TENSORBOARD_LOGDIR_LASSO + '/', metric_name="F1")
/opt/python/envs/default/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
/opt/python/envs/default/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:292: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-11-03 17:38:37,637] A new study created in RDB with name: lasso_regression
/opt/python/envs/default/lib/python3.8/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
/opt/pytho

FrozenTrial(number=14, values=[0.9605457661525151], datetime_start=datetime.datetime(2022, 11, 3, 17, 38, 45, 102716), datetime_complete=datetime.datetime(2022, 11, 3, 17, 38, 45, 429614), params={'alpha': 0.0016053955926854306, 'fit_intercept': True, 'scaler': 0}, distributions={'alpha': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'fit_intercept': CategoricalDistribution(choices=(True, False)), 'scaler': CategoricalDistribution(choices=(0, 1, 2, 3))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=15, state=TrialState.COMPLETE, value=None)

Let us check out the result of the parameter search:

In [16]:
study.best_params

{'alpha': 0.0016053955926854306, 'fit_intercept': True, 'scaler': 0}

In order to evaluate the real performance the test split is given to the model:

In [17]:
scaler = scalers[study.best_params['scaler']]

lasso = Lasso(alpha=study.best_params['alpha'], fit_intercept=study.best_params['fit_intercept'])
lasso.fit(df_no_id(all_X_train), binarize_response(df_no_id(all_y_train), False))
result_dict['F1']['Lasso'] = measure_dict['F1'](list(binarize_response(all_y_test, False)[RESPONSE_COLUMN]), apply_threshold_on_response(list(lasso.predict(df_no_id(all_X_test))), False))
result_dict['F1']['Lasso']

/opt/python/envs/default/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.391e+01, tolerance: 4.259e-02
  model = cd_fast.enet_coordinate_descent(


0.9382716049382716

This is surprisingly good. We use this as the baseline now. It is important to use something simple as baseline, as it is otherwise very hard to determine whether the neural network that we will build in the next chapter is performing below expectations and has therefore likely a bug in the code.

## The shallow neural network

For our next trick we are going to use a shallow neural network.

However before we get started, the pytorch class "Dataset" will have to be adapted for our use case.

In [18]:
from torch.utils.data import Dataset


class CancerDataset(Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        super().__init__()
        self.__ids = X['id']
        self.__X = df_no_id(X.copy())
        self.__y = df_no_id(binarize_response(y.copy(), True))

    def __getitem__(self, index):
        return np.array(self.__X.iloc[index].values), np.array(self.__y.iloc[index].values)

    def __len__(self):
        return len(self.__X)

This code might seem a bit messy - it would seem way cleaner to have the trial object suggest parameters outside of the neural network class and instantiate the network with these parameters passed in the constructor. However consider following situation:

```
if trial.suggest_float('test_1', 0, 1) > 0.5:
    # code that may or may not be executed
    do_something(trial.suggest_int('test_2', 0, 100)
```

By generating the values right where they are needed, the risk is eliminated of having optuna mistakenly assume effects of parameters that are not even used. The code below might also look more complicated than it needs to be for now. However essentially what happens here is that we let optuna decide how deep the network is and how many neurons there are per layer. Of course the first input size is the number of values that go into the network and the last input size is the number of values that should be predicted. Lastly dropout are added as regularizers. They can help e.g. to make neural networks less dependent on single columns in the dataset.

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader

class NeuralNetwork(nn.Module):
    def __init__(self, input_size: int, trial: optuna.Trial):
        super(NeuralNetwork, self).__init__()
        temp = []
        last_input = input_size
        for i in range(n_layers := trial.suggest_int('n_layers', 1, 10)):
            is_final_layer = i == n_layers - 1
            if is_final_layer:
                linear_out = 2
            else:
                linear_out = trial.suggest_int(f'linear_{i}_out', 2 , 2500, log=True)

            temp.append(nn.Linear(last_input, linear_out))
            last_input = linear_out

            if not is_final_layer:
                temp.append(nn.Dropout(p=trial.suggest_float(f'dropout_{i}', 0, 0.5)))
                temp.append(nn.LeakyReLU())

        self.linear_leaky_relu_stack = nn.Sequential(
            *temp
        )

    def forward(self, x):
        logits = self.linear_leaky_relu_stack(x)
        return nn.functional.softmax(logits, 1) # 1 is the dimension
    

In the next cell loops for training and testing are created. Note that using tensors in python wrappers such as lists and tuples may cause problems with memory. Also make sure to detach tensors before you collect them, so that the memory can be freed again.

In [20]:
# from memory_profiler import profile

# @profile # comment this line in if you are having trouble with memory usage
def train_loop(dataloader, model, loss_fn, optim, print_progress = False):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(iter(dataloader)):
        # Compute prediction and loss
        pred = model(X.type(torch.float).to(DEVICE))
        y = torch.tensor(np.stack(np.array(tuple(expand_classes(cur_y.item()) for cur_y in y))), dtype=torch.float).to(DEVICE)
        optim.zero_grad()
        loss = loss_fn(pred, y)
        loss.backward()
        optim.step()

        if print_progress and batch % 100 == 0:
            loss, current = loss.detach().item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        possible_emergency_brake()
        cpu_cooldown()


def test_loop(dataloader, model, report = False):
    model.eval()
    f1_scores_test = []
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.type(torch.float).to(DEVICE))
            cur_score = f1_score([torch.argmax(t.detach()).detach().item() for t in pred], [t.detach().item() for t in y])
            f1_scores_test.append(cur_score)

    result = sum(f1_scores_test) / len(f1_scores_test)
    if report:
        print(f"Test f1-score: = {result}")
    return result

Now let us find the optimal parameters. Note that numpy arrays are used to store tensors instead of lists or tuples. This is preferable for reasons regarding memory management. If you are interested (or having problems with memory), you may read up on that.

This code below allows that the inputs are either boundaries for a selection (e.g. batch size from 1 to 100 or a list of scalers to choose from) or concrete values. This means that we can reuse this method later when we have fixed some of the choices and are still testing for some others. We can also leverage this method for the final testing by fixing all of the values! The bit of additional complexity therefore really pays off.

In [34]:
def test_params(trial: Optional[optuna.Trial], datasets: Union[Tuple[pd.DataFrame, pd.DataFrame], List[Tuple[pd.DataFrame, pd.DataFrame]], CancerDataset], model_type: type, scalers: Union[List[object], object], batch_size: Union[Tuple[int, int], int], loss_functions: Union[List[Callable], Callable], learning_rate: Union[Tuple[float, float], float], prune: bool) -> (float, int):
    nn_f1_scores = []
    best_epochs = []

    if isinstance(datasets, tuple):
        datasets = [datasets]
    for cur_seed, d in enumerate(datasets):
        torch.manual_seed(cur_seed)
        
        k_train_X, k_train_y = split_to_xy(d[0])
        if isinstance(scalers, list):
            scaler = scalers[trial.suggest_categorical(name='scaler', choices=list(range(len(scalers))))]
        else:
            scaler = scalers
        
        k_train_X = pd.DataFrame(scaler.fit_transform(k_train_X.to_numpy()), columns=k_train_X.columns)
        k_test_X, k_test_y = split_to_xy(d[1])
        k_test_X = pd.DataFrame(scaler.fit_transform(k_test_X.to_numpy()), columns=k_train_X.columns)

        if isinstance(batch_size, tuple):
            assert len(batch_size) == 2
            assert batch_size[0] < batch_size[1]
            batch_size = trial.suggest_int('batch_size', batch_size[0], batch_size[1])
        

        if isinstance(loss_functions, list):
            loss_fn = loss_functions[trial.suggest_categorical('loss_fn', list(range(len(loss_functions))))]
        else:
            loss_fn = loss_functions

        if isinstance(learning_rate, tuple):
            assert learning_rate[0] < learning_rate[1]
            cur_learning_rate = trial.suggest_float('lr', 1e-6, 1e-3, log=True)
        else:
            cur_learning_rate = learning_rate

        if trial is None:
            model = model_type(all_X_train.shape[1] - 1).to(DEVICE)
        else:
            model = model_type(all_X_train.shape[1] - 1, trial).to(DEVICE)

        opt = torch.optim.Adam(lr=cur_learning_rate, params=[p for p in model.parameters() if p.requires_grad])

        cur_train_dataset = CancerDataset(k_train_X, k_train_y)
        tr_loader = DataLoader(cur_train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
        cur_test_dataset = CancerDataset(k_test_X, k_test_y)

        best_result: float = - np.inf
        best_epoch: int = -1

        for ep in range(TRAIN_FOR_EPOCHS):
            train_loop(tr_loader, model, loss_fn, opt)
            te_loader = DataLoader(cur_test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
            result = test_loop(te_loader, model)

            if result > best_result:
                best_result = result
                best_epoch = ep
            
            if trial is not None and prune:
                trial.report(result, ep)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

            nn_f1_scores.append(best_result)
            best_epochs.append(best_epoch)
        return sum(nn_f1_scores) / len(nn_f1_scores), best_epochs
    

def run_study(name: str, datasets: Union[Tuple[pd.DataFrame, pd.DataFrame], List[Tuple[pd.DataFrame, pd.DataFrame]], CancerDataset], model_type: Callable, scalers: Union[List[object], object], batch_size: Union[Tuple[int, int], int], loss_functions: Union[List[Callable], Callable], learning_rate: Union[Tuple[float, float], float], n_startup_trials: int, n_trials: int, prune: bool, tensorboard_cb: optuna.integration.TensorBoardCallback) -> optuna.Trial:
    try:
      study = optuna.load_study(storage=OPTUNA_DB, study_name=name)
    except KeyError:
        def objective(trial):
            return test_params(trial, datasets, model_type, scalers, batch_size, loss_functions, learning_rate, prune)[0]

        if prune:
            pruner = optuna.pruners.SuccessiveHalvingPruner()
        else:
            pruner = None
        study = optuna.create_study(storage=OPTUNA_DB, study_name=name, direction="maximize", sampler=optuna.samplers.TPESampler(seed=0, n_startup_trials=n_startup_trials, multivariate=True, group=True), load_if_exists=True,  pruner=pruner)

        study.optimize(objective, n_trials=n_trials, show_progress_bar=True, callbacks=[tensorboard_cb])
    return study

Note that pruning is NOT used here. The reason for this is that runs are not comparable in their first epochs. A deeper network might need more epochs of training in order to get to a good result.

In [22]:
import numpy as np
tensorboard_callback_nn = TensorBoardCallback(TENSORBOARD_LOGDIR_NN_SHALLOW + '/', metric_name="F1")
loss_functions = [nn.BCELoss(), nn.HingeEmbeddingLoss(), nn.KLDivLoss(reduction='batchmean')]
study = run_study('nn_shallow', datasets_tvt, NeuralNetwork, scalers, (1, 100), loss_functions, (1e-6, 1e-3), 250, 500, False, tensorboard_callback_nn)

[I 2022-11-03 17:39:37,971] Trial 0 finished with value: 0.13837024171370674 and parameters: {'scaler': 1, 'batch_size': 43, 'loss_fn': 2, 'lr': 0.0007780155576901416, 'n_layers': 4, 'linear_0_out': 533, 'dropout_0': 0.26444745987645224, 'linear_1_out': 101, 'dropout_1': 0.4627983191463305, 'linear_2_out': 3, 'dropout_2': 0.043564649850770354}. Best is trial 0 with value: 0.13837024171370674.
[I 2022-11-03 17:39:39,757] Trial 1 finished with value: 0.0 and parameters: {'scaler': 3, 'batch_size': 98, 'loss_fn': 0, 'lr': 2.263722969739549e-06, 'n_layers': 7, 'linear_0_out': 4, 'dropout_0': 0.47233445852479194, 'linear_1_out': 72, 'dropout_1': 0.2073309699952618, 'linear_2_out': 11, 'dropout_2': 0.38711684471710833, 'linear_3_out': 44, 'dropout_3': 0.28421697443432425, 'linear_4_out': 2, 'dropout_4': 0.30881774853793853, 'linear_5_out': 141, 'dropout_5': 0.30846699843737846}. Best is trial 0 with value: 0.13837024171370674.
[I 2022-11-03 17:39:41,205] Trial 2 finished with value: 0.0 and 

<ipython-input-22-7439391e44c9>:2: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
  tensorboard_callback_nn = TensorBoardCallback(TENSORBOARD_LOGDIR_NN_SHALLOW + '/', metric_name="F1")
/opt/python/envs/default/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
/opt/python/envs/default/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:292: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-11-03 17:39:35,978] A new study created in RDB with name: nn_shallow
/opt/python/envs/default/lib/python3.8/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


Lets check the results of this:

In [23]:
study.best_params

{'batch_size': 70,
 'dropout_0': 0.41254567127451774,
 'dropout_1': 0.1841969635108051,
 'dropout_2': 0.4578606088564118,
 'dropout_3': 0.46765020419328074,
 'linear_0_out': 1180,
 'linear_1_out': 667,
 'linear_2_out': 2450,
 'linear_3_out': 316,
 'loss_fn': 1,
 'lr': 0.0009363244593836366,
 'n_layers': 5,
 'scaler': 2}

You can comment-in the next cell to take a look at the results of the parameter search. set the parameter to e.g. TENSORBOARD_LOGDIR_NN_SHALLOW if you want to view the board for the previous search of hyper parameters for the shallow neural network. Unfortunately switching back and forth between different boards only seem to work if you restart the notebooks kernel in between.

In [24]:
# view_tensorboard(TENSORBOARD_LOGDIR_NN_SHALLOW)

Alternatively, if you are running this code locally, you can enter localhost:x into your browser where x is the port given by the method call below.

In [47]:
# notebook.list()

In this cell the importance of the hyperparameters is calculated. Note that this graph will differ ever so slightly at each run because of some randomness. Unfortunately I was not able to find the right way to set a seed manually.

Quite surprisingly when I first ran this code the number of learnable model parameters (that is the number of weights the model has - here defined by linear_0_out and linear_1_out) almost did not matter! It is tempting to just disregard these variables now, but the error I made was in the assumption that the ideal parameter for the sizes of the layers lie in a much lower rates. As a reaction I changed these ranges and their importance in this graph went up.

(In the current state of the project these parameters cannot be seen in this graph anymore, since the depth of the network is now determined dynamically. However, I am sure that there are options to change this.)

In [26]:
optuna.visualization.plot_param_importances(study, evaluator=optuna.importance.FanovaImportanceEvaluator(n_trees=10, max_depth=8), target_name='F1')

Now let's see if the results calculated on the validation sets are comparable with those in the test set. If the test performance is significantly worse, then we might have accidentally used information of data from the validation split when training the model before. Pay special attention to aggregates of the dataset you use or metrics that were calculated over parts of the dataset.

In [53]:
best_params = study.best_params

class NeuralNetwork(nn.Module):
    def __init__(self, input_size: int):
        super(NeuralNetwork, self).__init__()
        temp = []
        last_input = input_size
        for i in range(n_layers := best_params['n_layers']):
            is_final_layer = i == n_layers - 1
            if is_final_layer:
                linear_out = 2
            else:
                linear_out = best_params[f'linear_{i}_out']

            temp.append(nn.Linear(last_input, linear_out))
            last_input = linear_out

            if not is_final_layer:
                temp.append(nn.Dropout(p=best_params[f'dropout_{i}']))
                temp.append(nn.LeakyReLU())

        self.linear_leaky_relu_stack = nn.Sequential(
            *temp
        )

    def forward(self, x):
        logits = self.linear_leaky_relu_stack(x)
        return nn.functional.softmax(logits, 1) # 1 is the dimension

# Reducing randomness of result by averaging accross multiple random seeds
scores = []
for i in range(10):
    torch.manual_seed(i)
    val_score, best_epochs = test_params(None, datasets_tvt, NeuralNetwork, scalers[best_params['scaler']], best_params['batch_size'], loss_functions[best_params['loss_fn']], best_params['lr'], False)

    final_model = NeuralNetwork(all_X_train.shape[1] - 1)
    training_set = pd.DataFrame(scalers[best_params['scaler']].fit_transform(all_X_train.to_numpy()), columns=all_X_train.columns)
    torch.manual_seed(i)
    train_loader = DataLoader(CancerDataset(training_set, all_y_train), batch_size=best_params['batch_size'], shuffle=True, drop_last=False)
    optimizer = torch.optim.Adam(lr=best_params['lr'], params=[p for p in final_model.parameters() if p.requires_grad])

    for epoch in range(sorted(best_epochs)[int(len(best_epochs) / 2)] + 1):
        train_loop(train_loader, final_model, loss_functions[best_params['loss_fn']], optimizer)


    scores.append(test_loop(
        DataLoader(
            CancerDataset(
                pd.DataFrame(scalers[best_params['scaler']].fit_transform(all_X_test.to_numpy()), columns=all_X_test.columns), all_y_test
            ), batch_size=best_params['batch_size'], shuffle=False, drop_last=False
        ),
        final_model)
    )

    torch.save(final_model.state_dict(), f'final_model_{i}')

print(f'Validation F1 score = {val_score}')

result_dict['F1']['NN_shallow'] = sum(scores) / len(scores)

result_dict

Validation F1 score = 0.9825190358209227


{'F1': {'Lasso': 0.9382716049382716, 'NN_shallow': 0.9397242982835083}}

For reasons of reproducibility, the final model and sets are saved. (This is for my personal use — Never process or open pickle files from someone else, as that is not secure.) Note that the saving of the models has already been done in the code cell above.

In [51]:
import pickle

with open('all_X_train', 'wb+') as f:
    pickle.dump(all_X_train, f)

with open('all_X_test', 'wb+') as f:
    pickle.dump(all_X_test, f)

In [52]:
# tracemalloc.stop()

The improvement over the simple lasso model is not as significant as one might have thought. Perhaps the data is not complex enough for neural networks to really play their strengths. It should also be said that the lasso model is fully optimized as well and it seemed to have handled this kind of task quite nicely. Whatever the case, it does not really matter though, as for this notebook the journey was the goal. I hope that you had as interesting a time reading this as I had making it.